# Final

In [1]:
def QuantaMental(x):
    to_screen = []
    for stock in x:
        try:
            df_test = dr.data.get_data_yahoo(stock, start = datetime.datetime.now() - datetime.timedelta(days=5*365), end = datetime.datetime.now())
            df_test = df_test.transpose()
            df_test = df_test.loc['Adj Close']
            df_test = df_test.transpose()
            df_test = pd.DataFrame(df_test)
            df_test.reset_index(level=0, inplace=True)
            df_test['Date'] = df_test['Date'].astype('datetime64')
            df_test.rename(columns = {'Date': 'date', 'Adj Close': 'adj_close'}, inplace = True)

            def calc_slope(x):
                slope = np.polyfit(range(len(x)), x, 1)[0]
                return slope
            
            edf = si.get_cash_flow(stock)
            edf = edf.loc['netIncome']
            edf = pd.DataFrame(edf)
            Earnings = (((edf['netIncome'][0]-edf['netIncome'][1])/edf['netIncome'][1])+((edf['netIncome'][1]-edf['netIncome'][2])/edf['netIncome'][2])+((edf['netIncome'][2]-edf['netIncome'][3])/edf['netIncome'][3]))/4

            df_test['STD_DEV'] = df_test['adj_close'].rolling(window=264).std()
    #creates a column for 264 day (1 year - weekends) moving average
            df_test['Moving_Average'] = df_test['adj_close'].rolling(window=264).mean()
    #creates a z-score column as it compares to the 264 day rolling average and STD_DEV
            df_test['Z-Score'] = ((df_test['adj_close']-df_test['adj_close'].rolling(window=264).mean().shift(1))/df_test['adj_close'].rolling(window=264).std(ddof=0).shift(1))
    #creates a rolling slope column
            df_test['Rolling_Slope'] = df_test.adj_close.rolling(1825, min_periods=2).apply(calc_slope)
    #creates an average earnings column
            df_test['Earnings'] = Earnings
    #creates a column which holds a TRUE value if the stock has a z-score of -1.5 or less
            df_test['Criteria_1'] = df_test['Z-Score'] <= -1.5
    #creates a column which holds a TRUE value if the 5y trendline has a positive slope
            df_test['Criteria_2'] = df_test['Rolling_Slope'] > 0
    #creates a column which holds True if the stock has a positive earnings growth average
            df_test['Criteria_3'] = df_test['Earnings'] > 0
    #to chop the end of the dataframe and only use true criteria from the past quarter so we're using relevent data
            df_test = df_test.tail(66)
    #creates a column to test if the criterias are true in the past month
            df_test['Final_Criteria'] = (df_test['Criteria_1'] & df_test['Criteria_2'] & df_test['Criteria_3']) == True
    #creates a column that holds a TRUE value if all above criteria are also TRUE WITHIN THE PAST QUARTER
            for i in df_test['Final_Criteria']:
                if i == True:
                    to_screen.append(stock)
                    break
        except:
            pass
        
    return to_screen

In [2]:
screened = QuantaMental(#si.tickers_sp500())
screened

SyntaxError: unexpected EOF while parsing (<ipython-input-2-54bf6b6e9a23>, line 2)